In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import time

import missingno

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

In [9]:
df1 = pd.read_csv('2024년_수비_스탯_연봉_데이터(포지션)최종.csv')
df1

,선수,팀,WAR,G,oWAR,dWAR,PA,ePA,AB,R,...,OBP,SLG,OPS,R/ePA,wRC+,pid,연도,기존pid,연봉(만원),포지션
0,노시환,한화 이글스,2.41,136,2.50,-0.08,601,597,526,88,...,0.356,0.454,0.810,0.145,106.5,141259,2024,14125,35000.0,3루수
1,페라자,한화 이글스,2.13,122,2.27,-0.14,522,518,455,75,...,0.364,0.486,0.850,0.158,116.7,161219,2024,16121,104000.0,좌익수
2,최재훈,한화 이글스,1.82,116,1.69,0.13,348,340,280,34,...,0.371,0.336,0.707,0.131,96.6,101709,2024,10170,60000.0,포수
3,황영묵,한화 이글스,1.39,123,1.38,0.01,389,386,349,52,...,0.365,0.372,0.737,0.129,95.1,161109,2024,16110,3000.0,2루수
4,안치홍,한화 이글스,1.33,128,1.65,-0.31,533,531,473,64,...,0.370,0.427,0.797,0.146,107.9,100149,2024,10014,50000.0,1루수
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,고영우,키움 히어로즈,-0.42,94,0.01,-0.43,290,286,254,21,...,0.329,0.319,0.648,0.094,73.5,161287,2024,16128,3000.0,3루수
293,이원석,키움 히어로즈,-0.53,39,-0.33,-0.20,109,109,91,5,...,0.330,0.264,0.594,0.085,66.4,101807,2024,10180,40000.0,1루수
294,이승원,키움 히어로즈,-0.74,20,-0.53,-0.20,41,39,37,3,...,0.154,0.135,0.289,-0.058,-43.9,154767,2024,15476,3100.0,유격수
295,임지열,키움 히어로즈,-0.87,22,-0.59,-0.28,57,57,49,4,...,0.211,0.163,0.374,-0.011,-8.3,112257,2024,11225,7200.0,우익수


In [11]:
df2 = pd.read_csv('투수_스탯_연봉_파생변수(금액).csv')
df2

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,WHIP,WAR,팀,pid,연도,연봉(만원),신인여부,국적,선발투수,연봉등급
0,페냐,32,32,0,0,0,0,11,11,0,...,1.17,4.28,한화 이글스,15146,2024,65000,False,외국인,1,A
1,문동주,23,23,0,0,0,0,8,8,0,...,1.31,3.08,한화 이글스,15013,2024,10000,False,내국인,1,B
2,산체스,24,24,0,0,0,0,7,8,0,...,1.30,2.59,한화 이글스,15643,2024,50000,False,외국인,1,A
3,주현상,55,0,55,9,0,0,2,2,0,...,0.84,2.44,한화 이글스,11415,2024,11000,False,내국인,0,B
4,이태양,50,12,38,6,0,0,3,3,0,...,1.21,1.78,한화 이글스,10609,2024,50000,False,내국인,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,박범준,5,0,5,3,0,0,0,1,0,...,2.06,-0.19,키움 히어로즈,16133,2025,7100,True,내국인,0,C
422,전준표,18,5,13,4,0,0,2,4,0,...,1.99,-0.35,키움 히어로즈,16122,2025,3200,False,내국인,0,D
423,오석주,17,0,17,7,0,0,1,1,0,...,2.29,-0.50,키움 히어로즈,12860,2025,3900,False,내국인,0,D
424,조영건,25,3,22,3,0,0,2,1,0,...,1.91,-0.55,키움 히어로즈,14132,2025,3800,False,내국인,0,D


In [13]:
# 연도가 2023 인 컬럼
pitcher_2024 = df2[df2['연도'] == 2024]
pitcher_2024

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,WHIP,WAR,팀,pid,연도,연봉(만원),신인여부,국적,선발투수,연봉등급
0,페냐,32,32,0,0,0,0,11,11,0,...,1.17,4.28,한화 이글스,15146,2024,65000,False,외국인,1,A
1,문동주,23,23,0,0,0,0,8,8,0,...,1.31,3.08,한화 이글스,15013,2024,10000,False,내국인,1,B
2,산체스,24,24,0,0,0,0,7,8,0,...,1.30,2.59,한화 이글스,15643,2024,50000,False,외국인,1,A
3,주현상,55,0,55,9,0,0,2,2,0,...,0.84,2.44,한화 이글스,11415,2024,11000,False,내국인,0,B
4,이태양,50,12,38,6,0,0,3,3,0,...,1.21,1.78,한화 이글스,10609,2024,50000,False,내국인,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,이종민,11,2,9,4,0,0,0,1,0,...,2.05,-0.29,키움 히어로즈,14576,2024,3200,False,내국인,0,D
202,김동규,3,1,2,0,0,0,0,1,0,...,3.75,-0.47,키움 히어로즈,15485,2024,3100,False,내국인,0,D
203,김동혁,35,2,33,11,0,0,1,7,0,...,1.75,-0.51,키움 히어로즈,14674,2024,6000,False,내국인,0,C
204,하영민,57,0,57,7,0,0,3,1,0,...,1.76,-0.70,키움 히어로즈,11222,2024,8000,False,내국인,0,C


In [15]:
# 1. 타자 데이터프레임: 포지션구분 컬럼만 추가
df1['포지션구분'] = '타자'

# 2. 투수 데이터프레임: 포지션구분 컬럼만 추가
pitcher_2024['포지션구분'] = '투수'

# 3. 컬럼이 다를 수 있으므로 outer join 방식으로 병합
df_combined = pd.concat([df1, pitcher_2024], ignore_index=True, sort=False)

In [17]:
df_combined

,선수,팀,WAR,G,oWAR,dWAR,PA,ePA,AB,R,...,TBF,BK,WP,ERA,FIP,WHIP,신인여부,국적,선발투수,연봉등급
0,노시환,한화 이글스,2.41,136,2.50,-0.08,601.0,597.0,526.0,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,페라자,한화 이글스,2.13,122,2.27,-0.14,522.0,518.0,455.0,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,최재훈,한화 이글스,1.82,116,1.69,0.13,348.0,340.0,280.0,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,황영묵,한화 이글스,1.39,123,1.38,0.01,389.0,386.0,349.0,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,안치홍,한화 이글스,1.33,128,1.65,-0.31,533.0,531.0,473.0,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,이종민,키움 히어로즈,-0.29,11,NaN,NaN,NaN,NaN,NaN,13,...,72.0,0.0,0.0,7.24,3.61,2.05,False,내국인,0.0,D
499,김동규,키움 히어로즈,-0.47,3,NaN,NaN,NaN,NaN,NaN,10,...,25.0,0.0,4.0,22.50,13.95,3.75,False,내국인,0.0,D
500,김동혁,키움 히어로즈,-0.51,35,NaN,NaN,NaN,NaN,NaN,34,...,192.0,2.0,0.0,7.32,4.52,1.75,False,내국인,0.0,C
501,하영민,키움 히어로즈,-0.70,57,NaN,NaN,NaN,NaN,NaN,36,...,247.0,0.0,6.0,4.64,3.86,1.76,False,내국인,0.0,C


In [19]:
df1.columns

Index(['선수', '팀', 'WAR', 'G', 'oWAR', 'dWAR', 'PA', 'ePA', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO',
       'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'R/ePA', 'wRC+', 'pid',
       '연도', '기존pid', '연봉(만원)', '포지션', '포지션구분'],
      dtype='object')

In [21]:
pitcher_2024.columns

Index(['선수', 'G', 'GS', 'GR', 'GF', 'CG', 'SHO', 'W', 'L', 'S', 'HD', 'IP',
       'ER', 'R', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'HP', 'SO', 'BK', 'WP',
       'ERA', 'FIP', 'WHIP', 'WAR', '팀', 'pid', '연도', '연봉(만원)', '신인여부', '국적',
       '선발투수', '연봉등급', '포지션구분'],
      dtype='object')

In [26]:
pitcher_2024['포지션'] = '투수'

In [28]:
pitcher_2024

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,팀,pid,연도,연봉(만원),신인여부,국적,선발투수,연봉등급,포지션구분,포지션
0,페냐,32,32,0,0,0,0,11,11,0,...,한화 이글스,15146,2024,65000,False,외국인,1,A,투수,투수
1,문동주,23,23,0,0,0,0,8,8,0,...,한화 이글스,15013,2024,10000,False,내국인,1,B,투수,투수
2,산체스,24,24,0,0,0,0,7,8,0,...,한화 이글스,15643,2024,50000,False,외국인,1,A,투수,투수
3,주현상,55,0,55,9,0,0,2,2,0,...,한화 이글스,11415,2024,11000,False,내국인,0,B,투수,투수
4,이태양,50,12,38,6,0,0,3,3,0,...,한화 이글스,10609,2024,50000,False,내국인,0,A,투수,투수
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,이종민,11,2,9,4,0,0,0,1,0,...,키움 히어로즈,14576,2024,3200,False,내국인,0,D,투수,투수
202,김동규,3,1,2,0,0,0,0,1,0,...,키움 히어로즈,15485,2024,3100,False,내국인,0,D,투수,투수
203,김동혁,35,2,33,11,0,0,1,7,0,...,키움 히어로즈,14674,2024,6000,False,내국인,0,C,투수,투수
204,하영민,57,0,57,7,0,0,3,1,0,...,키움 히어로즈,11222,2024,8000,False,내국인,0,C,투수,투수


In [30]:



# 필요한 컬럼만 추출
df1 = df1[['선수', '팀', 'WAR', '연도', '포지션', '연봉(만원)','pid']]
pitcher_2024 = pitcher_2024[['선수', '팀', 'WAR', '연도', '포지션', '연봉(만원)','pid']]

# 포지션구분 컬럼 추가
df1['포지션구분'] = '타자'
pitcher_2024['포지션구분'] = '투수'

# 열 순서 맞추기
common_columns = ['선수', '팀', '연도', '포지션', '포지션구분', 'WAR', '연봉(만원)']
df1 = df1[common_columns]
pitcher_2024 = pitcher_2024[common_columns]

# 병합
merged_df = pd.concat([df1, pitcher_2024], ignore_index=True)


In [32]:
merged_df

,선수,팀,연도,포지션,포지션구분,WAR,연봉(만원)
0,노시환,한화 이글스,2024,3루수,타자,2.41,35000.0
1,페라자,한화 이글스,2024,좌익수,타자,2.13,104000.0
2,최재훈,한화 이글스,2024,포수,타자,1.82,60000.0
3,황영묵,한화 이글스,2024,2루수,타자,1.39,3000.0
4,안치홍,한화 이글스,2024,1루수,타자,1.33,50000.0
...,...,...,...,...,...,...,...
498,이종민,키움 히어로즈,2024,투수,투수,-0.29,3200.0
499,김동규,키움 히어로즈,2024,투수,투수,-0.47,3100.0
500,김동혁,키움 히어로즈,2024,투수,투수,-0.51,6000.0
501,하영민,키움 히어로즈,2024,투수,투수,-0.70,8000.0


In [34]:
merged_df.to_csv("2024_타자_투수_통합.csv", index=False)


In [36]:
df_2023 = pd.read_csv('2023_타자_투수_통합.csv')
df_2023

,선수,팀,연도,포지션,포지션구분,WAR,연봉(만원)
0,노시환,한화 이글스,2023,3루수,타자,6.53,13100.0
1,최재훈,한화 이글스,2023,포수,타자,3.23,80000.0
2,이진영,한화 이글스,2023,우익수,타자,2.44,3900.0
3,채은성,한화 이글스,2023,1루수,타자,1.79,180000.0
4,이도윤,한화 이글스,2023,유격수,타자,1.64,3400.0
...,...,...,...,...,...,...,...
540,김동규,키움 히어로즈,2023,투수,투수,-0.47,3000.0
541,김동혁,키움 히어로즈,2023,투수,투수,-0.51,6500.0
542,하영민,키움 히어로즈,2023,투수,투수,-0.70,6500.0
543,양현,키움 히어로즈,2023,투수,투수,-0.74,8500.0


In [38]:
df_2024 = pd.read_csv('2024_타자_투수_통합.csv')
df_2024

,선수,팀,연도,포지션,포지션구분,WAR,연봉(만원)
0,노시환,한화 이글스,2024,3루수,타자,2.41,35000.0
1,페라자,한화 이글스,2024,좌익수,타자,2.13,104000.0
2,최재훈,한화 이글스,2024,포수,타자,1.82,60000.0
3,황영묵,한화 이글스,2024,2루수,타자,1.39,3000.0
4,안치홍,한화 이글스,2024,1루수,타자,1.33,50000.0
...,...,...,...,...,...,...,...
498,이종민,키움 히어로즈,2024,투수,투수,-0.29,3200.0
499,김동규,키움 히어로즈,2024,투수,투수,-0.47,3100.0
500,김동혁,키움 히어로즈,2024,투수,투수,-0.51,6000.0
501,하영민,키움 히어로즈,2024,투수,투수,-0.70,8000.0


In [40]:
# 두 데이터를 그대로 이어붙이기 (컬럼 동일하므로 concat 사용)
df_all = pd.concat([df_2023, df_2024], ignore_index=True)

In [42]:
df_all

,선수,팀,연도,포지션,포지션구분,WAR,연봉(만원)
0,노시환,한화 이글스,2023,3루수,타자,6.53,13100.0
1,최재훈,한화 이글스,2023,포수,타자,3.23,80000.0
2,이진영,한화 이글스,2023,우익수,타자,2.44,3900.0
3,채은성,한화 이글스,2023,1루수,타자,1.79,180000.0
4,이도윤,한화 이글스,2023,유격수,타자,1.64,3400.0
...,...,...,...,...,...,...,...
1043,이종민,키움 히어로즈,2024,투수,투수,-0.29,3200.0
1044,김동규,키움 히어로즈,2024,투수,투수,-0.47,3100.0
1045,김동혁,키움 히어로즈,2024,투수,투수,-0.51,6000.0
1046,하영민,키움 히어로즈,2024,투수,투수,-0.70,8000.0


In [44]:
df_all.to_csv("2324_타자_투수_통합.csv", index=False)


In [46]:
bat2025 = pd.read_csv('2025년_수비_스탯_연봉_데이터.csv')
bat2025

,선수,팀,WAR,G,oWAR,dWAR,PA,ePA,AB,R,...,AVG,OBP,SLG,OPS,R/ePA,wRC+,pid,연도,기존pid,연봉(만원)
0,노시환,한화 이글스,2.41,136,2.50,-0.08,601,597,526,88,...,0.272,0.356,0.454,0.810,0.145,106.5,141259,2024,14125,33000.0
1,최재훈,한화 이글스,1.82,116,1.69,0.13,348,340,280,34,...,0.257,0.371,0.336,0.707,0.131,96.6,101709,2024,10170,50000.0
2,황영묵,한화 이글스,1.39,123,1.38,0.01,389,386,349,52,...,0.301,0.365,0.372,0.737,0.129,95.1,161109,2024,16110,8300.0
3,안치홍,한화 이글스,1.33,128,1.65,-0.31,533,531,473,64,...,0.300,0.370,0.427,0.797,0.146,107.9,100149,2024,10014,50000.0
4,최인호,한화 이글스,1.18,82,0.86,0.32,244,240,210,37,...,0.286,0.367,0.395,0.762,0.139,102.2,147079,2024,14707,6800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,고영우,키움 히어로즈,-0.42,94,0.01,-0.43,290,286,254,21,...,0.256,0.329,0.319,0.648,0.094,73.5,161287,2024,16128,4500.0
239,이원석,키움 히어로즈,-0.53,39,-0.33,-0.20,109,109,91,5,...,0.220,0.330,0.264,0.594,0.085,66.4,101807,2024,10180,30000.0
240,이승원,키움 히어로즈,-0.74,20,-0.53,-0.20,41,39,37,3,...,0.108,0.154,0.135,0.289,-0.058,-43.9,154767,2024,15476,3200.0
241,임지열,키움 히어로즈,-0.87,22,-0.59,-0.28,57,57,49,4,...,0.102,0.211,0.163,0.374,-0.011,-8.3,112257,2024,11225,5800.0
